# Tugas Besar A
IF3270 Pembelajaran Mesin<br>
Forward Propagation - Feed Forward Neural Network (FFNN)

Developed by:
1. K01 13520010 - Ken Kalang Al Qalyubi
2. K01 13520036 - I Gede Arya Raditya Parameswara
3. K02 13520061 - Gibran Darmawan
4. K03 13520119 - Marchotridyo

## Main Program

### Library

In [19]:
import json, math
from enum import Enum

### Enum

In [20]:
class LayerEnum(Enum):
    INPUT = "INPUT"
    HIDDEN = "HIDDEN"
    OUTPUT = "OUTPUT"

class ActivationFuncEnum(Enum):
    SIGMOID = "SIGMOID"
    LINEAR = "LINEAR"
    RELU = "RELU"
    SOFTMAX = "SOFTMAX"

### File Utility

In [21]:
class FileUtility:
    @staticmethod
    def import_json(file_name):
        with open(file_name) as json_file:
            return json.load(json_file)

    @staticmethod
    def export_json(file_name, data):
        with open(file_name, 'w') as outfile:
            json.dump(data, outfile)

### Neuron

In [22]:
class Neuron:
    def __init__(
        self, 
        layer_type: str, 
        activation_func: str, 
        weight: list,
        bias: list, 
    ):
        self.__layer_type: LayerEnum = layer_type
        self.__activation_func: ActivationFuncEnum = activation_func
        self.__weight: list = weight
        self.__bias: list = bias
        self.__net: float = 0.0
        self.__value: float = 0.0

    def activate(self):
        if self.__activation_func == ActivationFuncEnum.SIGMOID.value:
            self.__value = 1 / (1 + math.exp(-self.__net))
        elif self.__activation_func == ActivationFuncEnum.LINEAR.value:
            self.__value = self.__net
        elif self.__activation_func == ActivationFuncEnum.RELU.value:
            self.__value = max(0, self.__net)
        elif self.__activation_func == ActivationFuncEnum.SOFTMAX.value:
            # TODO: Ini harusnya mengakses semua .__net dari neuron pada layer yang sama
            self.__value = math.exp(self.__net) / sum([math.exp(net) for net in self.__net])

    def set_value(self, value):
        self.__value = value

    def set_net(self, net):
        self.__net = net

    def get_value(self):
        return self.__value

    def get_net(self):
        return self.__net

    def get_weight(self, index):
        return self.__weight[index]

    def get_weights(self):
        return self.__weight

    def get_bias(self):
        return self.__bias

    def get_layer_type(self):
        return self.__layer_type

    def get_activation_func(self):
        return self.__activation_func

### ANN Graph

In [23]:
class ANNGraph:
    def __init__(self, file_config_path: str):
        self.file_path = file_config_path
        self.config = None

        self.layers = []
        self.build_ann_graph()

    def build_ann_graph(self):
        self.config = FileUtility.import_json(self.file_path)
        layers = self.config["layers"]

        for layer in layers:
            current_layer = []

            for i in range(layer["total_neurons"]):
                neuron = self.__generate_neuron_data(layer, i)
                current_layer.append(neuron)

            self.layers.append(current_layer)

        return

    def predict(self, input_data):
        for i in range(len(input_data)):
            # Masukkan input_data ke neuron di input layer
            neuron: Neuron = self.layers[0][i]
            neuron.set_value(input_data[i])

        self.__activate_all_neurons()

        return

    def print_details(self):
        for i in range(len(self.layers)):
            print("--------------------")
            print("Layer", i+1)
            print("--------------------")
            for j in range(len(self.layers[i])):
                neuron: Neuron = self.layers[i][j]
                if j == 0:
                    print("Activation Function:", neuron.get_activation_func())
                print(f"[Neuron {j+1}] Net:", neuron.get_net())
                print(f"[Neuron {j+1}] Value:", neuron.get_value())
            print("")

    def __generate_neuron_data(self, layer, i):
        layer_type: str = layer["type"]
        activation_func: str = None
        weight: list = []
        bias: list = []

        if (layer_type == LayerEnum.HIDDEN.value or layer_type == LayerEnum.OUTPUT.value):
            activation_func = layer["activation_func"]
            weight = layer["weight"][i]
            bias = layer["bias"][i]

        return Neuron(layer_type, activation_func, weight, bias)
    
    def __activate_all_neurons(self):
        for i in range(1, len(self.layers)):
            for j in range(len(self.layers[i])):
                neuron: Neuron = self.layers[i][j]
                previous_layer = self.layers[i - 1]

                self.__generate_neuron_net(neuron, previous_layer)
                neuron.activate()

        return

    def __generate_neuron_net(self, neuron: Neuron, previous_layer: list):
        total = 0.0

        for i in range(len(previous_layer)):
            total += previous_layer[i].get_value() * neuron.get_weight(i)

        # TODO: Tambahkan weight untuk bias
        total += neuron.get_bias()
        neuron.set_net(total)

## Testing

### Sigmoid

In [24]:
graph = ANNGraph("config/xor_sigmoid.json")
graph.predict([0, 0])
graph.print_details()

--------------------
Layer 1
--------------------
Activation Function: None
[Neuron 1] Net: 0.0
[Neuron 1] Value: 0
[Neuron 2] Net: 0.0
[Neuron 2] Value: 0

--------------------
Layer 2
--------------------
Activation Function: SIGMOID
[Neuron 1] Net: -10.0
[Neuron 1] Value: 4.5397868702434395e-05
[Neuron 2] Net: 30.0
[Neuron 2] Value: 0.9999999999999065

--------------------
Layer 3
--------------------
Activation Function: SIGMOID
[Neuron 1] Net: -9.999092042627819
[Neuron 1] Value: 4.543910487654591e-05

